In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils

In [3]:
parsed_symptoms = pathutils.get_data_file("04_06_new_data/parsed/symptoms_04_16_5.csv.csv")
raw_symptoms = pathutils.get_data_file("04_06_new_data/data/symptoms_04_16_5.csv")

In [4]:
import scipy.sparse as sparse

/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [39]:
import pandas as pd
import numpy as np

In [18]:
parsed_df = pd.read_csv(parsed_symptoms)

In [19]:
parsed_df = parsed_df.drop(columns=['Unnamed: 0'])

In [20]:
labels = parsed_df.LABEL
parsed_df = parsed_df.drop(columns=['LABEL'])

In [21]:
parsed_values = parsed_df.values

In [28]:
label_values = labels.values

In [22]:
parsed_sparse = sparse.lil_matrix(parsed_values)

In [27]:
csr_sparse = parsed_sparse.tocsr()

In [24]:
# train on the data ??
from sklearn.ensemble import RandomForestClassifier

/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/teliov/Library/anaconda3/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/teliov/Library/anaconda3/envs/ml/lib/pyth

In [25]:
clf = RandomForestClassifier(n_estimators=140, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=2, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [26]:
from sklearn.model_selection import StratifiedShuffleSplit

In [31]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(csr_sparse, label_values):
    train_data = csr_sparse[train_index]
    train_labels = label_values[train_index]
    test_data = csr_sparse[test_index]
    test_labels = label_values[test_index]

In [36]:
res = clf.fit(train_data, train_labels)

In [40]:
train_predictions = res.predict(train_data)
diff = (train_predictions - train_labels) != 0
num_missed = np.sum(diff)
num_labels = len(train_predictions)
accuracy = (num_labels - num_missed)*1.0/num_labels

In [41]:
print("Train Accuracy: %.3f" % accuracy)

Train Accuracy: 0.997


In [42]:
test_pred = clf.predict(test_data)
diff = (test_pred - test_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_pred)
accuracy = (num_labels - num_missed)*1.0/num_labels

In [43]:
print("Test Accuracy: %.3f" % accuracy)

Test Accuracy: 0.833


In [44]:
# how about results without the sparsity!
df = pd.read_csv(parsed_symptoms)

In [19]:
df = df.drop(columns=['Unnamed: 0'])

In [45]:
labels = df.LABEL
df = df.drop(columns=['LABEL'])

In [46]:
clf = RandomForestClassifier(n_estimators=140, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=2, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [47]:
train_df = df.iloc[train_index]
tr_labels = labels.iloc[train_index]

In [48]:
test_df = df.iloc[test_index]
tst_labels = labels.iloc[test_index]

In [49]:
res = clf.fit(train_df, tr_labels)

In [51]:
train_predictions = clf.predict(train_df)
diff = (train_predictions - tr_labels) != 0
num_missed = np.sum(diff)
num_labels = len(train_predictions)
accuracy = (num_labels - num_missed)*1.0/num_labels
print("Train Accuracy: %.3f" % accuracy)

Train Accuracy: 1.000


In [52]:
test_pred = clf.predict(test_df)
diff = (test_pred - tst_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_pred)
accuracy = (num_labels - num_missed)*1.0/num_labels
print("Test Accuracy: %.3f" % accuracy)

Test Accuracy: 0.846


Accuracy drops when using the sparse representation of the matrix ..
Does it get better if we use csc  instead ??

In [53]:
csc_df = pd.read_csv(parsed_symptoms)

In [54]:
csc_df = csc_df.drop(columns=['Unnamed: 0'])

In [55]:
csc_labels = csc_df.LABEL
csc_df = csc_df.drop(columns=['LABEL'])

In [56]:
csc_values = csc_df.values

In [57]:
csc_label_values = csc_labels.values

In [58]:
csc_sparse = sparse.lil_matrix(csc_values)

In [60]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(csc_sparse, csc_label_values):
    train_data = csc_sparse[train_index]
    train_labels = csc_label_values[train_index]
    test_data = csc_sparse[test_index]
    test_labels = csc_label_values[test_index]

In [61]:
train_data = train_data.tocsc()
test_data = test_data.tocsc()

In [62]:
res = clf.fit(train_data, train_labels)

In [63]:
train_predictions = res.predict(train_data)
diff = (train_predictions - train_labels) != 0
num_missed = np.sum(diff)
num_labels = len(train_predictions)
accuracy = (num_labels - num_missed)*1.0/num_labels

In [64]:
print("Train Accuracy: %.3f" % accuracy)

Train Accuracy: 0.997


In [65]:
test_pred = clf.predict(test_data)
diff = (test_pred - test_labels) != 0
num_missed = np.sum(diff)
num_labels = len(test_pred)
accuracy = (num_labels - num_missed)*1.0/num_labels

In [66]:
print("Test Accuracy: %.3f" % accuracy)

Test Accuracy: 0.844


In [121]:
l = {
    'a': [1, 2, 3],
    'b': [5, 6, 7],
    'c': [[1,2,3], [4], [2]]
}
p = pd.DataFrame(l)

In [122]:
dense = p[['a', 'b']]

In [127]:
def flatten(item, init=[]):
    init += item
    return init

In [129]:
columns = []
_ = p.c.aggregate(flatten, init=columns)

In [131]:
df_size = p.c.aggregate(len)

In [135]:
import itertools

In [141]:
row = []
for idx, item in enumerate(df_size.__iter__()):
    row += [idx] * item

In [143]:
row, columns

([0, 0, 0, 1, 2], [1, 2, 3, 4, 2])

In [144]:
data = np.ones(len(row))

In [145]:
data

array([1., 1., 1., 1., 1.])

In [149]:
coo_symptoms = sparse.coo_matrix((data, (row, columns)), shape=(p.shape[0], 5))

In [151]:
coo_dense = sparse.coo_matrix(dense.values)

In [153]:
coo_comb = sparse.hstack([coo_dense, coo_symptoms])

In [154]:
coo_comb.toarray()

array([[1., 5., 0., 1., 1., 1., 0.],
       [2., 6., 0., 0., 0., 0., 1.],
       [3., 7., 0., 0., 1., 0., 0.]])

In [158]:
csc_comb = coo_comb.tocsc()

In [162]:
csc_comb[:, 0:2].toarray()

array([[1., 5.],
       [2., 6.],
       [3., 7.]])